In [2]:
from obspy import read_events
import pandas as pd
from datetime import datetime

In [4]:
file_path = "/home/developer/Desktop/NOTEBOOK/GMS/cgm-gm/PhaseNet/phase_picks.txt"

catalog = read_events(file_path, format="GSE2")

rows = []
for event in catalog:
    eid = str(event.resource_id)

    for origin in event.origins:
        src_lat = origin.latitude
        src_lon = origin.longitude 
        depth = origin.depth / 1000 if origin.depth else None 
        mag = event.magnitudes[0].mag if event.magnitudes else None  

        pick_times = [pick.time for pick in event.picks if pick.time]
        start_time = min(pick_times) if pick_times else None
        end_time = max(pick_times) if pick_times else None

        for pick in event.picks:
            station_code = pick.waveform_id.station_code if pick.waveform_id else None

            try:
                sampling_frequency = pick.extra.get("sampling_frequency", None)
            except AttributeError:
                sampling_frequency = 0

            try:
                num_time_steps = pick.extra.get("number_of_time_steps", None)
            except AttributeError:
                num_time_steps = 0

            try:
                delta_t = pick.extra.get("delta_t", None)
            except AttributeError:
                delta_t = 0

            try:
                min_frequency = pick.extra.get("min_frequency", None)
            except AttributeError:
                min_frequency = 0

            try:
                max_frequency = pick.extra.get("max_frequency", None)
            except AttributeError:
                max_frequency = 0

            try:
                num_frequency_steps = pick.extra.get("number_of_frequency_steps", None)
            except AttributeError:
                num_frequency_steps = 0

            rows.append({
                "eid": eid,
                "src_lat": src_lat,
                "src_lon": src_lon,
                "stat_lat": 0,
                "stat_lon": 0,
                "depth": depth,
                "mag": mag,
                "station_code": station_code,
                "start_time": start_time.isoformat() if start_time else None,  
                "end_time": end_time.isoformat() if end_time else None, 
                "Sampling Frequency": sampling_frequency,
                "Number of Time Steps": num_time_steps,
                "Delta t": delta_t,
                "Min Frequency": min_frequency,
                "Max Frequency": max_frequency,
                "Number of Frequency Steps": num_frequency_steps,
                "rup" : 0
            })

df = pd.DataFrame(rows)

df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")

filter_start = datetime(2023, 2, 6)
filter_end = datetime(2023, 2, 8)

filtered_df = df[(df["start_time"] >= filter_start) & (df["start_time"] < filter_end)]


output_csv = "filtered_phase_picks_feb_6_7.csv"
filtered_df.to_csv(output_csv, index=False)

print(f"Filtered CSV saved as {output_csv}")


Filtered CSV saved as filtered_phase_picks_feb_6_7.csv


In [5]:
!pwd

/home/developer/Desktop/NOTEBOOK/GMS/cgm-gm/notebooks
